### Librairie nécéssaire

In [1]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer as ps
import gensim
from gensim import corpora,models

### importotation des commentaires aprés scraping de la page h24info sur facebook

In [2]:
df=pd.read_csv("H24Infos.csv")
documents=df['text']

In [3]:
documents

0      Ils se sont rendus au port de Casablanca et so...
1      Mohammed Turk sera bel et bien entendu dans le...
2      L’année 2019 a été riche au niveau de la scène...
3                        Tous corps et grades confondus.
4      L’année 2019 a été marquée par la disparition ...
                             ...                        
730                                                 a.ma
731    Le #StudioCode30 by #CIHBank met à disposition...
732    sa dernière visite remonte à 400 ans et on ne ...
733    Nombreux parmi nous vont s'y reconnaître. Mais...
734    CAN 2017 (Maroc-Togo): Qui a été selon vous l'...
Name: text, Length: 735, dtype: object

### Processing du text

In [4]:
documents[2]

'L’année 2019 a été riche au niveau de la scène culturelle.'

In [5]:
documents[2].lower()

'l’année 2019 a été riche au niveau de la scène culturelle.'

In [6]:
text=[]
for d in documents:
    text.append(str(d).lower())

In [7]:
print(text)

['ils se sont rendus au port de casablanca et sont montés dans un bateau sans savoir quelle direction il prenait.', 'mohammed turk sera bel et bien entendu dans le cadre de l’affaire hamzamonbb.', 'l’année 2019 a été riche au niveau de la scène culturelle.', 'tous corps et grades confondus.', 'l’année 2019 a été marquée par la disparition de plusieurs personnalités marocaines qui ont contribué, chacun dans son domaine, au rayonnement du royaume.', "le parquet a fini par prendre au sérieux les soupçons d'implication directe des deux soeurs.", 'il est déjà au cœur d’un procès après avoir souhaité «qu’il y ait moins de marocains» aux pays-bas.', 'une enquête a été ouverte.', 'le café-restaurant présente une décoration très fleurie et «girly».', 'le cese indique également que le cadre légal, "pensé avec les technologies d\'hier", constitue une entrave à l\'innovation.', 'le droit à l’avortement, le respect des libertés individuelles et de la liberté d’expression, le droit à l’euthanasie… v

In [8]:
text_clean=[re.sub('\w*\d\w*','',text[i]) for i in range(len(text))]
text_clean

['ils se sont rendus au port de casablanca et sont montés dans un bateau sans savoir quelle direction il prenait.',
 'mohammed turk sera bel et bien entendu dans le cadre de l’affaire hamzamonbb.',
 'l’année  a été riche au niveau de la scène culturelle.',
 'tous corps et grades confondus.',
 'l’année  a été marquée par la disparition de plusieurs personnalités marocaines qui ont contribué, chacun dans son domaine, au rayonnement du royaume.',
 "le parquet a fini par prendre au sérieux les soupçons d'implication directe des deux soeurs.",
 'il est déjà au cœur d’un procès après avoir souhaité «qu’il y ait moins de marocains» aux pays-bas.',
 'une enquête a été ouverte.',
 'le café-restaurant présente une décoration très fleurie et «girly».',
 'le cese indique également que le cadre légal, "pensé avec les technologies d\'hier", constitue une entrave à l\'innovation.',
 'le droit à l’avortement, le respect des libertés individuelles et de la liberté d’expression, le droit à l’euthanasie…

In [9]:
tokenizer=RegexpTokenizer(r'\w+')

#### Tokenization

In [10]:
tokens=[tokenizer.tokenize(text_clean[i]) for i in range(len(text_clean))]
tokens

[['ils',
  'se',
  'sont',
  'rendus',
  'au',
  'port',
  'de',
  'casablanca',
  'et',
  'sont',
  'montés',
  'dans',
  'un',
  'bateau',
  'sans',
  'savoir',
  'quelle',
  'direction',
  'il',
  'prenait'],
 ['mohammed',
  'turk',
  'sera',
  'bel',
  'et',
  'bien',
  'entendu',
  'dans',
  'le',
  'cadre',
  'de',
  'l',
  'affaire',
  'hamzamonbb'],
 ['l',
  'année',
  'a',
  'été',
  'riche',
  'au',
  'niveau',
  'de',
  'la',
  'scène',
  'culturelle'],
 ['tous', 'corps', 'et', 'grades', 'confondus'],
 ['l',
  'année',
  'a',
  'été',
  'marquée',
  'par',
  'la',
  'disparition',
  'de',
  'plusieurs',
  'personnalités',
  'marocaines',
  'qui',
  'ont',
  'contribué',
  'chacun',
  'dans',
  'son',
  'domaine',
  'au',
  'rayonnement',
  'du',
  'royaume'],
 ['le',
  'parquet',
  'a',
  'fini',
  'par',
  'prendre',
  'au',
  'sérieux',
  'les',
  'soupçons',
  'd',
  'implication',
  'directe',
  'des',
  'deux',
  'soeurs'],
 ['il',
  'est',
  'déjà',
  'au',
  'cœur',
 

In [11]:
tokens_clean=[[t for t in token if t not in stopwords.words('french')] for token in tokens]

In [12]:
tokens_clean

[['rendus',
  'port',
  'casablanca',
  'montés',
  'bateau',
  'sans',
  'savoir',
  'quelle',
  'direction',
  'prenait'],
 ['mohammed',
  'turk',
  'bel',
  'bien',
  'entendu',
  'cadre',
  'affaire',
  'hamzamonbb'],
 ['année', 'a', 'riche', 'niveau', 'scène', 'culturelle'],
 ['tous', 'corps', 'grades', 'confondus'],
 ['année',
  'a',
  'marquée',
  'disparition',
  'plusieurs',
  'personnalités',
  'marocaines',
  'contribué',
  'chacun',
  'domaine',
  'rayonnement',
  'royaume'],
 ['parquet',
  'a',
  'fini',
  'prendre',
  'sérieux',
  'soupçons',
  'implication',
  'directe',
  'deux',
  'soeurs'],
 ['déjà',
  'cœur',
  'procès',
  'après',
  'avoir',
  'souhaité',
  'moins',
  'marocains',
  'pays',
  'bas'],
 ['enquête', 'a', 'ouverte'],
 ['café', 'restaurant', 'présente', 'décoration', 'très', 'fleurie', 'girly'],
 ['cese',
  'indique',
  'également',
  'cadre',
  'légal',
  'pensé',
  'technologies',
  'hier',
  'constitue',
  'entrave',
  'innovation'],
 ['droit',
  'avo

In [13]:
tokens_clean=[[t for t in token if len(t)!=1] for token in tokens_clean]

In [14]:
tokens_clean

[['rendus',
  'port',
  'casablanca',
  'montés',
  'bateau',
  'sans',
  'savoir',
  'quelle',
  'direction',
  'prenait'],
 ['mohammed',
  'turk',
  'bel',
  'bien',
  'entendu',
  'cadre',
  'affaire',
  'hamzamonbb'],
 ['année', 'riche', 'niveau', 'scène', 'culturelle'],
 ['tous', 'corps', 'grades', 'confondus'],
 ['année',
  'marquée',
  'disparition',
  'plusieurs',
  'personnalités',
  'marocaines',
  'contribué',
  'chacun',
  'domaine',
  'rayonnement',
  'royaume'],
 ['parquet',
  'fini',
  'prendre',
  'sérieux',
  'soupçons',
  'implication',
  'directe',
  'deux',
  'soeurs'],
 ['déjà',
  'cœur',
  'procès',
  'après',
  'avoir',
  'souhaité',
  'moins',
  'marocains',
  'pays',
  'bas'],
 ['enquête', 'ouverte'],
 ['café', 'restaurant', 'présente', 'décoration', 'très', 'fleurie', 'girly'],
 ['cese',
  'indique',
  'également',
  'cadre',
  'légal',
  'pensé',
  'technologies',
  'hier',
  'constitue',
  'entrave',
  'innovation'],
 ['droit',
  'avortement',
  'respect',
 

#### Stemming

In [15]:
stemmer=ps()
stems=[[stemmer.stem(t) for t in token ]for token in tokens_clean]
print(stems)

[['rendu', 'port', 'casablanca', 'monté', 'bateau', 'san', 'savoir', 'quell', 'direct', 'prenait'], ['moham', 'turk', 'bel', 'bien', 'entendu', 'cadr', 'affair', 'hamzamonbb'], ['anné', 'rich', 'niveau', 'scène', 'culturel'], ['tou', 'corp', 'grade', 'confondu'], ['anné', 'marqué', 'disparit', 'plusieur', 'personnalité', 'marocain', 'contribué', 'chacun', 'domain', 'rayonn', 'royaum'], ['parquet', 'fini', 'prendr', 'sérieux', 'soupçon', 'implic', 'direct', 'deux', 'soeur'], ['déjà', 'cœur', 'procè', 'aprè', 'avoir', 'souhaité', 'moin', 'marocain', 'pay', 'ba'], ['enquêt', 'ouvert'], ['café', 'restaur', 'présent', 'décorat', 'trè', 'fleuri', 'girli'], ['cese', 'indiqu', 'également', 'cadr', 'légal', 'pensé', 'technolog', 'hier', 'constitu', 'entrav', 'innov'], ['droit', 'avort', 'respect', 'liberté', 'individuel', 'liberté', 'express', 'droit', 'euthanasi', 'voici', 'quelqu', 'une', 'thématiqu', 'secoué', 'société', 'marocain', 'tout', 'long', 'anné', 'écoulé'], ['alert', 'info', 'douni

### Manipulation du dictionnaire

In [16]:
dictionary=corpora.Dictionary(stems)
print(dictionary)
print(dictionary.num_docs)

Dictionary(3361 unique tokens: ['bateau', 'casablanca', 'direct', 'monté', 'port']...)
735


In [17]:
corpus = [dictionary.doc2bow(stem) for stem in stems]

In [18]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)],
 [(18, 1), (19, 1), (20, 1), (21, 1), (22, 1)],
 [(23, 1), (24, 1), (25, 1), (26, 1)],
 [(18, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1)],
 [(2, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1)],
 [(31, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1)],
 [(54, 1), (55, 1)],
 [(56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1)],
 [(13, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1)],
 [(18, 1),
  (31, 1),
  (73, 1),
  (74, 2),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1)

### Application du LDA

In [19]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word = dictionary, passes=20)

In [20]:
topics=ldamodel.print_topics(num_topics=-1)

In [21]:
topics

[(0,
  '0.024*"dernier" + 0.020*"peux" + 0.020*"fraud" + 0.010*"plusieur" + 0.010*"artist" + 0.010*"mahjoub" + 0.010*"moham" + 0.010*"an" + 0.010*"famil" + 0.010*"raji"'),
 (1,
  '0.022*"arrivé" + 0.022*"mercredi" + 0.022*"villa" + 0.016*"plu" + 0.011*"attaqu" + 0.011*"prolifiqu" + 0.011*"term" + 0.011*"mettait" + 0.011*"annoncé" + 0.011*"espagnol"'),
 (2,
  '0.027*"bonn" + 0.018*"maroc" + 0.018*"franc" + 0.018*"entr" + 0.018*"el" + 0.018*"tiendra" + 0.018*"pari" + 0.011*"marocain" + 0.009*"indémod" + 0.009*"selham"'),
 (3,
  '0.082*"vote" + 0.021*"inchangé" + 0.021*"mise" + 0.021*"indiqu" + 0.021*"rest" + 0.021*"déjà" + 0.021*"opératoir" + 0.021*"comité" + 0.021*"mode" + 0.021*"plateform"'),
 (4,
  '0.055*"françai" + 0.034*"maroc" + 0.028*"jeudi" + 0.028*"accordé" + 0.028*"casablanca" + 0.014*"dimanch" + 0.014*"airbnb" + 0.014*"justic" + 0.014*"face" + 0.014*"hôtelier"'),
 (5,
  '0.018*"deux" + 0.018*"techniqu" + 0.016*"marocain" + 0.012*"depui" + 0.012*"aprè" + 0.012*"plusieur" + 0.0